In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

In [5]:
historical_defects = pd.read_csv('data/historical_defects.csv')
inspection_sessions = pd.read_csv('data/inspection_sessions.csv')
defects = pd.read_csv('data/defects.csv')
train_defects = pd.read_csv('data/train_defects.csv', delimiter=',')

In [7]:
train_data = train_defects.merge(defects, on="defect_id")
train_data = train_data.rename(columns = {'repeat_defect_detected':'label'})
features = train_data.drop('label', axis=1).columns

In [13]:
train_data1 = train_data.merge(inspection_sessions, on='inspection_id')

In [20]:
# we can consider dropping these two columns inspection_number: # of inspections on machine
train_data1.groupby(['inspection_id','defect_id']).count()

defect_sequence_no  label  defect_subcategory_id  \
inspection_id defect_id                                                     
1             10246                       1      1                      1   
              11109                       1      1                      1   
              13176                       1      1                      1   
              22035                       1      1                      1   
              43633                       1      1                      1   
...                                     ...    ...                    ...   
3421063       35548                       1      1                      1   
              49235                       1      1                      1   
3421070       4724                        1      1                      1   
              16953                       1      1                      1   
              35951                       1      1                      1   

                         defect_category_id  machine_id  eval_set  \
inspection_id defect_id                                             
1             10246                       1           1         1   
              11109                       1           1         1   
              13176                       1           1         1   
              22035                       1           1         1   
              43633                       1           1         1   
...                                     ...         ...       ...   
3421063       35548                       1           1         1   
              49235                       1           1         1   
3421070       4724                        1           1         1   
              16953                       1           1         1   
              35951                       1           1         1   

                         inspection_number  inspection_dow  \
inspection_id defect_id                                      
1             10246                      1               1   
              11109                      1               1   
              13176                      1               1   
              22035                      1               1   
              43633                      1               1   
...                                    ...             ...   
3421063       35548                      1               1   
              49235                      1               1   
3421070       4724                       1               1   
              16953                      1               1   
              35951                      1               1   

                         inspection_hour_of_day  days_since_last_inspection  
inspection_id defect_id                                                      
1             10246                           1                           1  
              11109                           1                           1  
              13176                           1                           1  
              22035                           1                           1  
              43633                           1                           1  
...                                         ...                         ...  
3421063       35548                           1                           1  
              49235                           1                           1  
3421070       4724                            1                           1  
              16953                           1                           1  
              35951                           1                           1  

[1384617 rows x 10 columns]

In [11]:
inspection_sessions[inspection_sessions['eval_set']=='test']

,inspection_id,machine_id,eval_set,inspection_number,inspection_dow,inspection_hour_of_day,days_since_last_inspection
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0
...,...,...,...,...,...,...,...
3420918,2728930,206202,test,23,2,17,6.0
3420929,350108,206204,test,5,4,14,14.0
3421001,1043943,206206,test,68,0,20,0.0
3421018,2821651,206207,test,17,2,13,14.0


In [9]:
x_train = train_data[features].values
y_train = train_data['label'].values

In [2]:
rf_model = RandomForestRegressor(n_estimators=5000, oob_score=True, random_state=100)
rf_model.fit(x_train, y_train)
pred_train = rf_model.predict(x_train)

print(np.sqrt(mean_squared_error(y_train,pred_train)))
print(r2_score(y_train, pred_train))

